In [6]:
import sqlite3
import threading
import time


In [7]:
connection = sqlite3.connect('acid_demo.db', check_same_thread=False)
connection.execute("PRAGMA foreign_keys = ON;")
lock = threading.Lock()

In [8]:
def setup_database():
    with connection:
        connection.execute("DROP TABLE IF EXISTS accounts;")
        connection.execute("CREATE TABLE accounts (id INTEGER PRIMARY KEY, name TEXT, balance REAL CHECK(balance >= 0));")
        connection.execute("INSERT INTO accounts (name, balance) VALUES ('Alice', 1000.0), ('Bob', 500.0);")


def demonstrate_atomicity():
    try:
        with connection:
            connection.execute("UPDATE accounts SET balance = balance - 200 WHERE name = 'Alice';")
            
            connection.execute("UPDATE accounts SET balance = balance + 200 WHERE name = 'Charlie';")
    except sqlite3.Error as e:
        print("Atomicity Violation - Transaction Rolled Back:", e)


def demonstrate_consistency():
    try:
        with connection:
            connection.execute("UPDATE accounts SET balance = -100 WHERE name = 'Alice';")
    except sqlite3.Error as e:
        print("Consistency Violation - Transaction Aborted:", e)


def demonstrate_isolation():
    def transaction_1():
        with lock:
            local_conn = sqlite3.connect('acid_demo.db')
            local_conn.execute("BEGIN;")
            local_conn.execute("UPDATE accounts SET balance = balance - 300 WHERE name = 'Alice';")
            time.sleep(2)
            local_conn.execute("COMMIT;")
            local_conn.close()

    def transaction_2():
        with lock:
            local_conn = sqlite3.connect('acid_demo.db')
            local_conn.execute("BEGIN;")
            local_conn.execute("UPDATE accounts SET balance = balance + 300 WHERE name = 'Bob';")
            local_conn.execute("COMMIT;")
            local_conn.close()

    thread1 = threading.Thread(target=transaction_1)
    thread2 = threading.Thread(target=transaction_2)
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()


def demonstrate_durability():
    with connection:
        connection.execute("UPDATE accounts SET balance = balance + 100 WHERE name = 'Alice';")
    # Reopen connection to verify
    connection.close()
    durable_conn = sqlite3.connect('acid_demo.db')
    cursor = durable_conn.execute("SELECT * FROM accounts;")
    for row in cursor:
        print(row)
    durable_conn.close()


def check_balances():
    cursor = connection.execute("SELECT * FROM accounts;")
    for row in cursor:
        print(row)



In [9]:
# Run demo
setup_database()
print("Balances before transactions:")
check_balances()

demonstrate_atomicity()
print("\nAfter Atomicity Demonstration:")
check_balances()

demonstrate_consistency()
print("\nAfter Consistency Demonstration:")
check_balances()

demonstrate_isolation()
print("\nAfter Isolation Demonstration:")
check_balances()

demonstrate_durability()
print("\nAfter Durability Demonstration:")


Balances before transactions:
(1, 'Alice', 1000.0)
(2, 'Bob', 500.0)

After Atomicity Demonstration:
(1, 'Alice', 800.0)
(2, 'Bob', 500.0)
Consistency Violation - Transaction Aborted: CHECK constraint failed: balance >= 0

After Consistency Demonstration:
(1, 'Alice', 800.0)
(2, 'Bob', 500.0)

After Isolation Demonstration:
(1, 'Alice', 500.0)
(2, 'Bob', 800.0)
(1, 'Alice', 600.0)
(2, 'Bob', 800.0)

After Durability Demonstration:
